<a href="https://colab.research.google.com/github/liminovna/glossary_compiler/blob/main/glossary_oop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
1) принимаемые форматы:
  - pdf (распознанный)
  - epub
  - txt
  - docx
  - fb2
2) языки:
  - англ
  - русский
# ! добавить ocr для pdf
# djvu
# tf-idf или ppmi!!! пока только абсолютный порог или топ
# добавить возможность  составлять триграммы и возможность выбирать тип н-грамм
"""

In [98]:
# config
ALLOWED_EXTENSIONS = ('pdf', 'txt', 'epub', 'docx', 'fb2')

SUPPORTED_LANGUAGES = ('ru','en')

In [66]:
!pip install pymupdf

In [54]:
import pymupdf

In [114]:
def foo():
  for page in pymupdf.open('Alice_in_Wonderland.pdf'):
    yield page

a = foo()

In [116]:
for i in a:
  print(i.number)
  # break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76


In [57]:
next(a).get_text()

'ALICE’S ADVENTURES\nIN WONDERLAND\nby Lewis Carroll\nwith fourty-two illustrations by John Tenniel\nThis book is in public domain.\nNo rigths reserved. Free for copy and distribution.\nThis PDF book is designed and published by PDFREEBOOKS.ORG\n'

In [85]:
from razdel import tokenize, sentenize


def foo():
  for page in pymupdf.open('Alice_in_Wonderland.pdf'):
    yield page

a = foo()


s = next(a).get_text()
res = []

for _ in s.split('\n'):
  res.extend(list(sentenize(_)))
  # print(_)

res

[Substring(0, 18, 'ALICE’S ADVENTURES'),
 Substring(0, 13, 'IN WONDERLAND'),
 Substring(0, 16, 'by Lewis Carroll'),
 Substring(0, 45, 'with fourty-two illustrations by John Tenniel'),
 Substring(0, 30, 'This book is in public domain.'),
 Substring(0, 19, 'No rigths reserved.'),
 Substring(20, 51, 'Free for copy and distribution.'),
 Substring(0,
           58,
           'This PDF book is designed and published by PDFREEBOOKS.ORG'),
 Substring(0, 0, '')]

In [84]:
list(sentenize(next(a).get_text()))

[Substring(0,
           126,
           'ALICE’S ADVENTURES\nIN WONDERLAND\nby Lewis Carroll\nwith fourty-two illustrations by John Tenniel\nThis book is in public domain.'),
 Substring(127, 146, 'No rigths reserved.'),
 Substring(147, 178, 'Free for copy and distribution.'),
 Substring(179,
           237,
           'This PDF book is designed and published by PDFREEBOOKS.ORG')]

In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [

    'This is the first document.',

    'This book is the second document.',

    'And this is the third one.',

    'Is this the first document?',

]

vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(corpus)

vectorizer.get_feature_names_out()

array(['and', 'book', 'document', 'first', 'is', 'one', 'second', 'the',
       'third', 'this'], dtype=object)

In [96]:
import pandas as pd

df = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names_out())
print(df)

        and      book  document     first        is       one    second  \
0  0.000000  0.000000  0.469791  0.580286  0.384085  0.000000  0.000000   
1  0.000000  0.556901  0.355463  0.000000  0.290614  0.000000  0.556901   
2  0.511849  0.000000  0.000000  0.000000  0.267104  0.511849  0.000000   
3  0.000000  0.000000  0.469791  0.580286  0.384085  0.000000  0.000000   

        the     third      this  
0  0.384085  0.000000  0.384085  
1  0.290614  0.000000  0.290614  
2  0.267104  0.511849  0.267104  
3  0.384085  0.000000  0.384085  


In [64]:
from collections.abc import Iterator
from collections import Counter

class InvalidExtension(Exception):
  pass

class UnsupportedLanguage(Exception):
  pass

class NotYetImplemented(Exception):
  pass

from typing import Literal

class TextProcessorRU:
  from razdel import tokenize

  STOP_WORDS = []

  def __init__(self, text):
    self.text = text

  @staticmethod
  def separate_sents(text: str) -> list:
    sentences = []

    # предварительно разбиваем текст со страницы по \n
    for _ in text.split('\n'):
      sentences.extend(list(razdel.sentenize(_)))
    self.sentences = sentences
    return sentences

  @staticmethod
  def tokenize(sent: str) -> list:
    return list(razdel.tokenize(sent))

  @staticmethod
  def lemmatize(words: list) -> list:
    lemmas = []
    for w in words:
      pass
    return lemmas

  def count_lemmas(self, lemma_list):
    pass

class TextProcessorEN:
  STOP_WORDS = []

  def separate_sents(text):
    pass

  def tokenize(text) -> list:
    pass

  def lemmatize(word) -> str:
    pass

  def count_lemmas(lemma_list):
    pass

def TextProcessor(lang, text):
     counters = {
        "ru": TextProcessorRU,
        "en": TextProcessorEN
    }

    return counters[lang](text)

class GlossaryCounter():
  """
  s = 'Алисе надоело сидеть на пригорке рядом с сестрой и ничего не делать. Раза два она заглянула украдкой в книгу, которую читала её сестра, но там не было ни разговоров, ни картинок.'
  glossary = GlossaryCounter(s)
  """

  def __init__(self, pages: Iterator, ngram_types, lang: str):
    self.pages = pages
    self.ngram_types = ngram_types
    self.lang = lang

    self.vocabulary_uni = defaultdict(tuple)
    self.vocabulary_uni.default_factory = vocabulary_uni.__len__
    self.feature_counter_uni = Counter()

    self.location_dict = defaultdict(tuple)

    self.vocabulary_bi = defaultdict(tuple)
    self.vocabulary_bi.default_factory = vocabulary_bi.__len__
    self.feature_counter_bi = Counter()

  def preprocess(text) -> str:
    return text.lower()

  def extract(self):
    """
    For every page, do ...
    """
    processor = TextProcessor(self.lang)
    while True:
      try:
        current_page = next(self.pages)
        current_text = self.preprocess(current_page.get_text())

        for s in processor.separate_sents(current_text):
          if s != '':
            lemmas = [processor.lemmatize(t) for t in processor.tokenize(s)]

            for left_word_idx in range(len(lemmas)):
              right_word_idx = left_word_idx + 1

              left_w = lemmas[left_word_idx]
              if left_w not in processor.stop_words:

                feature_idx_uni = self.vocabulary_uni[left_w]

                self.feature_counter_uni.update(feature_idx_uni)

                if feature_idx_uni not in self.location_dict:
                    self.location_dict[feature_idx_uni] = [(current_page.number, left_word_idx)]
                else:
                    self.location_dict[feature_idx_uni] = self.location_dict[feature_idx_uni] + [(current_page.number, left_word_idx)]

                if right_word_idx <= len(lemmas):
                  right_w = lemmas[right_word_idx]
                  if right_w not in processor.stop_words
                    feature_idx_bi = self.vocabulary_bi[(left_w,right_w)]

                    self.feature_counter_bi.update(feature_idx_bi)

class IteratorsCollection:

  def iterate_file(filename='Alice_in_Wonderland.txt'):
    """
    Read txt, pdf, epub, mobi, docx, or fb2 file using pymupdf
    """
    for page in pymupdf.open(filename):
      yield page

  def iterate_djvu(filename):
    raise NotYetImplemented('This feature is yet to be implemented')

from collections import defaultdict

class GlossaryCompiler:

  """
  Методы класса:
  а) вытащить топ n монограм и биграм
  б) выбрать типы n-грамм
  в) выгрузить в csv или xlsx и указать название аутпута
  """

  @staticmethod
  def get_extension(filename: str) -> str:
    """
    >>> get_extension('Alice_in_Wonderland.txt')
    'txt'
    >>> get_extension('Катехизис.pdf')
    'pdf'
    """
    res = filename.split('.')[-1]
    if res in ALLOWED_EXTENSIONS:
      return res
    raise InvalidExtension('Could not determine the file extension or the file extension is not allowed')

  @staticmethod
  def check_language(s: str):
    if s.lower() in SUPPORTED_LANGUAGES:
      return s.lower()
    raise UnsupportedLanguage(f'Language {s} is not supported')

  def __init__(self, filename: str, lang: Literal[SUPPORTED_LANGUAGES], page_size: int=2000) -> None:
    self.filename = filename
    self.page_size = page_size
    self.extension = self.get_extension(filename.lower())
    self.lang = self.check_language(lang)

  def __repr__(self) -> str:
    return f'ready to process {self.extension.upper()}-file {self.filename}'

  def build_vocab(self, ngram_types: list[int] = [1,2]):
    # words_total = 0
    # loc_dict = dict() # {'lemma': ((1,15), (1,35))}
    # unigram_cntr = defaultdict(tuple)

    if self.extension in ('pdf', 'txt', 'epub', 'docx', 'fb2'):
      pages = IteratorsCollection.iterate_file(self.filename)
    elif self.extension == ('djvu'):
      pages = IteratorsCollection.iterate_djvu()

    self.vocab = GlossaryCounter(pages=self.pages, ngram_types=self.ngram_types, lang=self.lang)
    return self

  def compile_glossary(self, top_n, united=False, min_threshold='auto'):
    pass

inst = GlossaryCompiler(filename='Alice_in_Wonderland.txt', lang='RU')
inst.build_vocab()
inst.pass(top_n=100, )

txt


In [51]:
import array
from collections import defaultdict, Counter

stop_words = []

vocabulary_uni = defaultdict(tuple)
vocabulary_uni.default_factory = vocabulary_uni.__len__

vocabulary_bi = defaultdict(tuple)
vocabulary_bi.default_factory = vocabulary_bi.__len__

# values = array.array(str("i"))
# j_indices = []
# indptr = []

# indptr.append(0)
feature_counter_uni = Counter()
feature_counter_bi = Counter()

location_dict = defaultdict(tuple)

a = ['a', 'b', 'c', 'a', 'b', 'b']
found_page = 0
for l, k in zip(a, a[1:]+['']):
  print(found_page)
  if l not in stop_words:
    # print((l, k))
  # for l in ['a', 'b', 'c', 'a', 'b', 'b']:
    feature_idx_uni = vocabulary_uni[l]
    feature_idx_bi = vocabulary_bi[(l,k)]

    print((vocabulary_uni.items(), vocabulary_bi.items()))

    if feature_idx_uni not in feature_counter_uni:
        feature_counter_uni.update(feature_idx_uni)
    else:
        feature_counter_uni.update(feature_idx_uni)

    if feature_idx_bi not in feature_counter_bi:
        feature_counter_bi.update(feature_idx_bi)
    else:
        feature_counter_bi.update(feature_idx_bi)

    if feature_idx_uni not in location_dict:
        location_dict[feature_idx_uni] = ((0, 0))
    else:
        location_dict[feature_idx_uni] = location_dict[feature_idx_uni] + (found_page,)
  found_page += 1

# j_indices.extend(feature_counter.keys())
# values.extend(feature_counter.values())
# indptr.append(len(j_indices))


print((feature_counter_uni, vocabulary_uni.items()))
# print((feature_counter_bi, vocabulary_bi.items()))

0
(dict_items([('a', 0)]), dict_items([(('a', 'b'), 0)]))
0
(dict_items([('a', 0), ('b', 1)]), dict_items([(('a', 'b'), 0), (('b', 'c'), 1)]))
0
(dict_items([('a', 0), ('b', 1), ('c', 2)]), dict_items([(('a', 'b'), 0), (('b', 'c'), 1), (('c', 'a'), 2)]))
0
(dict_items([('a', 0), ('b', 1), ('c', 2)]), dict_items([(('a', 'b'), 0), (('b', 'c'), 1), (('c', 'a'), 2)]))
0
(dict_items([('a', 0), ('b', 1), ('c', 2)]), dict_items([(('a', 'b'), 0), (('b', 'c'), 1), (('c', 'a'), 2), (('b', 'b'), 3)]))
0
(dict_items([('a', 0), ('b', 1), ('c', 2)]), dict_items([(('a', 'b'), 0), (('b', 'c'), 1), (('c', 'a'), 2), (('b', 'b'), 3), (('b', ''), 4)]))
({0: 2, 1: 3, 2: 1}, dict_items([('a', 0), ('b', 1), ('c', 2)]))


In [52]:
location_dict

defaultdict(tuple, {0: (0, 0, 0), 1: (0, 0, 0, 0), 2: (0, 0)})

In [163]:
feature_counter

{0: 2, 1: 1, 2: 1}

In [162]:
vocabulary

defaultdict(<method-wrapper '__len__' of collections.defaultdict object at 0x7edaacd8dee0>,
            {'a': 0, 'b': 1, 'c': 2})

In [ ]:
for feature in analyze(doc):
    try:
        feature_idx = vocabulary[feature]
        if feature_idx not in feature_counter:
            feature_counter[feature_idx] = 1
        else:
            feature_counter[feature_idx] += 1
    except KeyError:
        # Ignore out-of-vocabulary items for fixed_vocab=True
        continue

    j_indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    indptr.append(len(j_indices))

In [128]:
vocabulary['a']
vocabulary

TypeError: 'tuple' object is not callable

In [119]:
from collections import defaultdict
a = defaultdict(tuple)
a['a']

()

In [ ]:
class GlossaryExporter:
  pass

In [57]:
FileReader(filename='Alice_in_Wonderland.txt', lang='RU')
# itr.extract_vocab(ngram_types=[1, 2, 3], threshold='auto', ) #threshold=True auto

UnsupportedLanguage: This language (RU) is not supported

In [6]:
'abcdefg'.rindex('f')

5

In [ ]:
!pip install pymupdf # https://pymupdf.readthedocs.io/en/latest/

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 64.5 MB/s eta 0:00:00


In [ ]:
!pip install razdel

In [ ]:
# Razdel
from razdel import tokenize as razdel_tokenize

' | '.join([token.text for token in list(razdel_tokenize('стены покрашены в бледно-желтый цвет'))])

'стены | покрашены | в | бледно-желтый | цвет'